# Experiments about efficiency of Agentic Class

In [3]:
import pyterrier as pt
import pyterrier_rag.readers
from pyterrier_rag.backend import HuggingFaceBackend

import agent_framework
from agent_framework import R1Searcher
from agent_framework import SearchR1

sparse_index = pt.Artifact.from_hf('pyterrier/ragwiki-terrier')
bm25_ret = pt.rewrite.tokenise() >> sparse_index.bm25(include_fields=['docno', 'text', 'title'], threads=5, verbose = True) >> pt.rewrite.reset() >> pt.debug.print_rows()

INFO 08-22 14:19:59 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-22 14:19:59 [__init__.py:239] Automatically detected platform cuda.


Java started (triggered by tokenise) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/opt/miniconda3/lib/python3.10/site-packages/pyterrier/terrier/retriever.py:219: UserWarning: Multi-threaded retrieval is experimental, YMMV.
  warn(
/opt/miniconda3/lib/python3.10/site-packages/pyterrier/terrier/retriever.py:226: UserWarning: Upgrading indexref /mnt/resources/pyterrier-cache/artifacts/ebfd80cc597a31719f11ab5cd11ad8f441bc460f760c82ff66413ba9fb06943f/data.properties to be concurrent
  warn(


In [4]:
r1_searcher_bm25 = R1Searcher(
    retriever = bm25_ret
)

INFO 08-22 14:20:32 [config.py:717] This model supports multiple tasks: {'reward', 'embed', 'classify', 'score', 'generate'}. Defaulting to 'generate'.
INFO 08-22 14:20:32 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 08-22 14:20:32 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 08-22 14:20:33 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='XXsongLALA/Qwen-2.5-7B-base-RAG-RL', speculative_config=None, tokenizer='XXsongLALA/Qwen-2.5-7B-base-RAG-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=5096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_conf

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 08-22 14:22:46 [loader.py:458] Loading weights took 131.78 seconds
INFO 08-22 14:22:47 [gpu_model_runner.py:1347] Model loading took 14.2717 GiB and 132.391053 seconds
INFO 08-22 14:22:49 [kv_cache_utils.py:634] GPU KV cache size: 120,096 tokens
INFO 08-22 14:22:49 [kv_cache_utils.py:637] Maximum concurrency for 5,096 tokens per request: 23.57x
INFO 08-22 14:22:49 [core.py:159] init engine (profile, create kv cache, warmup model) took 2.32 seconds
INFO 08-22 14:22:49 [core_client.py:439] Core engine process 0 ready.
[R1Searcher] vLLM backend ready: XXsongLALA/Qwen-2.5-7B-base-RAG-RL


In [3]:
dataset =  pt.get_dataset('rag:nq')
from ir_measures import define_byquery
Iterations = define_byquery(lambda qrels, run: run.iloc[0].iteration, name="Iterations")
pt.Experiment(
    [r1_searcher_bm25],
    dataset.get_topics('dev').head(20), # NB: remove .head(100) to run on all dev topics
    dataset.get_answers('dev'),
    [pyterrier_rag.measures.F1, pyterrier_rag.measures.EM, "mrt"],
    batch_size=1,
    verbose=True,
    names=['R1Searcher(BM25)']
)

pt.Experiment:   0%|          | 0/20 [00:00<?, ?batches/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


TerrierRetr(BM25): 100%|██████████| 1/1 [00:01<00:00,  1.18s/q]


,qid,docid,docno,text,title,rank,score,query
0,dev_0-1,2935744,2935744,"Baby Love (Nicole Scherzinger song) """"Baby Lov...","""Baby Love (Nicole Scherzinger song)""",0,25.715294,Does He Love Me song artist
1,dev_0-1,20740662,20740662,"Ghetto Love (Karl Wolf song) """"Ghetto Love"""" i...","""Ghetto Love (Karl Wolf song)""",1,25.692845,Does He Love Me song artist


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...


TerrierRetr(BM25): 100%|██████████| 1/1 [00:00<00:00,  3.11q/s]


,qid,docid,docno,text,title,rank,score,query
0,dev_19-1,18825163,18825163,fine of $500 and/or six months of jail time fo...,"""Manatee conservation""",0,48.117343,manatees endangered list
1,dev_19-1,199409,199409,Florida Fish & Wildlife Conservation Commissio...,"""West Indian manatee""",1,47.198478,manatees endangered list


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

pt.Experiment: 100%|██████████| 20/20 [01:16<00:00,  3.85s/batches]


,name,F1,EM,mrt
0,R1Searcher(BM25),0.383333,0.25,3844.694704


In [7]:
import pyterrier_dr

e5 = pyterrier_dr.E5() >> pt.Artifact.from_hf("pyterrier/ragwiki-e5.flex") >> sparse_index.text_loader(['docno', 'title', 'text'])

In [8]:
r1_searcher_e5 = R1Searcher(
    retriever = e5
)

INFO 08-22 14:27:44 [config.py:717] This model supports multiple tasks: {'reward', 'embed', 'classify', 'score', 'generate'}. Defaulting to 'generate'.
INFO 08-22 14:27:44 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 08-22 14:27:44 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 08-22 14:27:44 [utils.py:2382] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
WARNING 08-22 14:27:50 [utils.py:2522] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x7fdd62e441c0>
WARNING 08-22 14:27:51 [topk_topp_sampler.py:69] FlashInfer is not availabl

Process EngineCore_0:
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 400, in run_engine_core
    raise e
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 387, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 329, in __init__
    super().__init__(vllm_config, executor_class, log_stats,
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 64, in __init__
    self.model_executor = executor_class(vllm_config)
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/executor/executor_base.py", line 52, in __init__
    self

[R1Searcher] vLLM init failed, fallback to transformers: Engine core initialization failed. See root cause above.


/opt/miniconda3/lib/python3.10/site-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[R1Searcher] transformers backend ready: XXsongLALA/Qwen-2.5-7B-base-RAG-RL on cpu


In [9]:
dataset =  pt.get_dataset('rag:nq')
pt.Experiment(
    [r1_searcher_bm25, r1_searcher_e5],
    dataset.get_topics('dev').head(20), # NB: remove .head(100) to run on all dev topics
    dataset.get_answers('dev'),
    [pyterrier_rag.measures.F1, pyterrier_rag.measures.EM, "mrt"],
    batch_size=1,
    verbose=True,
    names=['R1Searcher(BM25)','R1Searcher(E5)']
)

pt.Experiment:   0%|          | 0/40 [00:00<?, ?batches/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


TerrierRetr(BM25): 100%|██████████| 1/1 [00:00<00:00,  1.10q/s]


,qid,docid,docno,text,title,rank,score,query
0,dev_0-1,2935744,2935744,"Baby Love (Nicole Scherzinger song) """"Baby Lov...","""Baby Love (Nicole Scherzinger song)""",0,25.715294,Does He Love Me song artist
1,dev_0-1,20740662,20740662,"Ghetto Love (Karl Wolf song) """"Ghetto Love"""" i...","""Ghetto Love (Karl Wolf song)""",1,25.692845,Does He Love Me song artist


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...


TerrierRetr(BM25): 100%|██████████| 1/1 [00:00<00:00,  3.66q/s]


,qid,docid,docno,text,title,rank,score,query
0,dev_19-1,18825163,18825163,fine of $500 and/or six months of jail time fo...,"""Manatee conservation""",0,48.117343,manatees endangered list
1,dev_19-1,199409,199409,Florida Fish & Wildlife Conservation Commissio...,"""West Indian manatee""",1,47.198478,manatees endangered list


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

NumpyRetriever scoring: 100%|██████████| 5131/5131 [00:01<00:00, 3409.35docbatch/s]

NumpyRetriever scoring: 100%|██████████| 5131/5131 [00:01<00:00, 3222.63docbatch/s]

pt.Experiment: 100%|██████████| 40/40 [1:18:44<00:00, 118.10s/batches]


,name,F1,EM,mrt
0,R1Searcher(BM25),0.302051,0.2,4155.578638
1,R1Searcher(E5),0.395238,0.3,232039.116478
